In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from pathlib import Path

import pytorch_lightning as pl

import torch
from torch import nn
from torch_geometric.nn import knn_graph
from torch_geometric import transforms
from torch_geometric.data import Data, DataLoader
from torchmetrics.functional import mean_squared_error

from gvp import SyntheticGNN, SyntheticGVP, SyntheticDataModule
from gvp.gvp import _split

In [3]:
data_dir = Path("../data/synthetic")

In [17]:
model = SyntheticGVP((1, 1), (20, 4), (1, 1), (20, 4))

transform = transforms.Compose([
    transforms.KNNGraph(k=10),
    transforms.Cartesian(),
    transforms.Distance(),
])

dm = SyntheticDataModule(data_dir, 32, "off_center", transform, num_workers=2)
dm.setup('fit')

# data0 = next(iter(dm.train_dataloader()))
# data0.x

In [18]:
for batch in dm.train_dataloader():
    h_V = _split(batch.x, 1)
    edge_index = batch.edge_index
    h_E = _split(batch.edge_attr, 1)
    out = model(h_V, edge_index, h_E, batch=batch.batch)
    print(out.shape)
    loss = mean_squared_error(out, batch.y)
    print(loss)
    break

torch.Size([32])
tensor(7.0538, grad_fn=<DivBackward0>)
